# Chapter 4: Extending Agents with Tools

Agents in Pydantic AI can perform a variety of tasks. However, sometimes, they need extra functionality, such as accessing live data, interacting with databases, or performing calculations. Tools allow agents to access external functions, making them more versatile and capable.

In this tutorial, we'll walk through how to extend agents with tools, covering:

- Creating and registering tools
- Using built-in tools
- Building powerful custom tools
- Dependency injection for modular design

We'll showcase these concepts with real-world examples instead of generic functions.

Before we start, we need to set up the environment.

In [2]:
import nest_asyncio
nest_asyncio.apply()

from pydantic_ai import Agent, RunContext
import os
import dotenv
import random

dotenv.load_dotenv()

# Set your Google API key
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

## 1. Introduction to Tools in Pydantic AI

Tools allow an agent to call functions whenever needed. They can be:

- Plain tools (`@agent.tool_plain`) → Standalone functions that return data
- Context-aware tools (`@agent.tool`) → Require additional input from the agent's context

We’ll demonstrate these in practical applications.

## 2. Registering Tools in an Agent

Let's create an AI-powered workout planner that suggests exercises based on user preferences and fitness levels.

In [15]:
# Initialize the agent
agent = Agent(
    'google-gla:gemini-2.0-flash',
    system_prompt="You are a fitness coach that suggests personalized workout plans.",
)

# Define a tool to suggest workouts
@agent.tool_plain
def suggest_workout(goal: str) -> str:
    """Suggests a workout based on the user's fitness goal."""
    workouts = {
        "strength": ["Deadlifts", "Squats", "Bench Press"],
        "cardio": ["Running", "Cycling", "Jump Rope"],
        "flexibility": ["Yoga", "Dynamic Stretching", "Pilates"],
    }
    
    # Check for partial matches in the goal
    for key in workouts:
        if key in goal.lower():
            return f"Try this workout for {key}: {random.choice(workouts[key])}"
    
    return f"Try this workout: {random.choice(['Rest Day', 'Walking', 'Light Stretching'])}"

# Run the agent
result = agent.run_sync('I want a workout for strength training.')
print(result.data)

I suggest you try Bench Press for strength training.



## 3. Using Built-in Tools

Pydantic AI comes with common tools like duckduckgo_search_tool() for web searches.

Install the duckduckgo tool if your environment doesn't have it.

In [ ]:
pip install 'pydantic-ai-slim[duckduckgo]' --quiet

Note: you may need to restart the kernel to use updated packages.


In [16]:
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool

# Initialize the agent with a built-in search tool
agent = Agent(
    'google-gla:gemini-1.5-flash',
    tools=[duckduckgo_search_tool()],
    system_prompt="Search DuckDuckGo for the given query and return the results.",
)

# Run the agent with a query
result = agent.run_sync('What is the current President of Malaysia?')
print(result.data)

The current Prime Minister of Malaysia is Anwar Ibrahim.  Malaysia has a parliamentary system; there is no President.



I found out that only the system prompt: `Search DuckDuckGo for the given query and return the results.` works, else it will return an error.

## 4. Dependency Injection in Tools

Pydantic AI supports dependency injection, allowing you to manage dependencies effectively within your tools. This feature is particularly useful for injecting services or configurations that your tools require.

In [20]:
# Initialize the agent
agent = Agent(
    'google-gla:gemini-2.0-flash',
    system_prompt='You provide exchange rate information to users.',
)

# Define a custom tool with dependency injection
@agent.tool
def get_exchange_rate(ctx: RunContext[dict], currency: str) -> str:
    """Fetch the exchange rate for a given currency."""
    exchange_rates = ctx.deps['exchange_rates']
    rate = exchange_rates.get(currency.upper(), 'unknown')
    return f"The exchange rate for {currency.upper()} is {rate}."

# Dependency data
dependencies = {
    'exchange_rates': {
        'USD': '1.00',
        'EUR': '0.85',
        'JPY': '110.00',
    }
}

# Run the agent with a user query
result = agent.run_sync('What is the exchange rate for EUR?', deps=dependencies)
print(result.data)

The exchange rate for EUR is 0.85.



## 5. Integrating Agents with External APIs

### Example: Real-Time Crypto Price Tracking

In [22]:
import requests

# Initialize the Crypto Agent
crypto_agent = Agent(
    'google-gla:gemini-1.5-flash',
    system_prompt='You provide real-time cryptocurrency prices and trends.',
)

# Define a tool to fetch Bitcoin price with CoinGecko API
@crypto_agent.tool
def get_bitcoin_price(ctx: RunContext) -> str:
    """Fetches the current price of Bitcoin and recent trend."""
    try:
        # Use CoinGecko API to get Bitcoin data for the last 7 days
        url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=7&interval=daily"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        # Extract prices
        prices = data['prices']
        
        # Get the most recent price
        current_price = prices[-1][1]
        
        # Calculate price change percentage over the period
        first_price = prices[0][1]
        price_change = ((current_price - first_price) / first_price) * 100
        
        # Format the response
        trend = "up" if price_change > 0 else "down"
        return f"The current price of Bitcoin is ${current_price:.2f} USD. " \
               f"Over the past week, the price has gone {trend} by {abs(price_change):.2f}%."
    
    except Exception as e:
        # Fallback to mock data when API is unavailable
        return f"Unable to fetch real-time Bitcoin price (Error: {type(e).__name__}). " \
               f"Using sample data: The current price of Bitcoin is $29,876.45 USD."

# Run the Crypto Agent
response = crypto_agent.run_sync('What is the current price of Bitcoin?')
print(response.data)

The current price of Bitcoin is $80186.92 USD.  Over the past week, the price has gone down by 16.59%.



## 6. Creating Advanced Custom Tools

### Example: Personal Finance AI

In [17]:
# Initialize the agent
agent = Agent(
    'google-gla:gemini-1.5-flash',
    system_prompt="You are a finance assistant that helps track expenses.",
)

# Define a tool with dependency injection
@agent.tool
def add_expense(ctx: RunContext[dict], category: str, amount: float) -> str:
    """Stores a user's expense in the system."""
    ctx.deps['expenses'].append({'category': category, 'amount': amount})
    return f"Added {amount} to {category} expenses."

# Initialize dependencies (storage for expenses)
dependencies = {'expenses': []}

# Run the agent
agent.run_sync('Add 50 to food expenses.', deps=dependencies)
agent.run_sync('Add 30 to transport expenses.', deps=dependencies)

# Print stored expenses
print(dependencies['expenses'])

[{'category': 'food', 'amount': 50.0}, {'category': 'transport', 'amount': 30.0}]


## 7. Combining Tools for Real-World Applications

In [18]:
# Initialize the agent
agent = Agent(
    'google-gla:gemini-1.5-flash',
    system_prompt="You recommend restaurants based on user preferences.",
)

# Tool to suggest a type of cuisine
@agent.tool_plain
def suggest_cuisine() -> str:
    """Suggests a random cuisine to try."""
    cuisines = ["Italian", "Japanese", "Mexican", "Indian", "Thai"]
    return random.choice(cuisines)

# Tool to fetch restaurants (simulating API response)
@agent.tool_plain
def find_restaurant(cuisine: str) -> str:
    """Finds a restaurant serving the specified cuisine."""
    restaurants = {
        "Italian": ["Pasta Heaven", "Luigi's Pizza"],
        "Japanese": ["Sushi World", "Ramen House"],
        "Mexican": ["Taco Land", "Burrito King"],
        "Indian": ["Spice Bazaar", "Curry Express"],
        "Thai": ["Bangkok Bites", "Thai Delight"],
    }
    return f"Try {random.choice(restaurants.get(cuisine, ['No options available']))} for {cuisine} food!"

# Run the agent
cuisine = suggest_cuisine()  # Get a random cuisine
result = find_restaurant(cuisine)  # Find a restaurant
print(f"{cuisine} cuisine → {result}")

Italian cuisine → Try Luigi's Pizza for Italian food!


- Multiple tools work together (suggest_cuisine + find_restaurant).
- This approach makes the agent modular and reusable.
- The logic can easily scale (e.g., integrating Google Places API).

## Conclusion

In this chapter, we explored how to extend Pydantic AI agents with tools.
We covered:
- Registering function tools (`@agent.tool` and `@agent.tool_plain`)
- Using built-in tools like `duckduckgo_search_tool()`
- Building custom tools for finance, fitness, crypto price tracking and food recommendations
- Dependency injection to store and manage agent state

With these skills, you can now create intelligent, real-world AI assistants

In the next chapter, we'll explore multi-agent systems, where multiple agents work together to achieve a common goal.